# Setup amazon forecast

In [1]:
%%capture --no-stderr setup

!pip install pandas s3fs matplotlib ipywidgets
!pip install boto3
!pip install boto3 --upgrade

%reload_ext autoreload

In [2]:
import sys
import os
import json
from time import sleep
from IPython.display import clear_output
import asyncio
import boto3
import s3fs
import pandas as pd

In [3]:
region = 'us-west-2'
bucket_name = ''
role_arn = ''
password_user = ''
password_user_secret = ''
user_iam = ''
url_user = ''

In [4]:
## This step is necessary to use the service of s3, iam, and other services

os.environ['AWS_ACCESS_KEY_ID'] = password_user
os.environ['AWS_SECRET_ACCESS_KEY'] = password_user_secret

In [5]:
## Configurations of the sessions and instances to forecast

session = boto3.Session(region_name=region, aws_access_key_id=password_user, aws_secret_access_key=password_user_secret)
forecast = session.client(service_name='forecast')
forecastQuery = session.client(service_name='forecastquery')

In [6]:
# Get all datasets used in your register 

datasets = forecast.list_datasets()

if datasets['Datasets']:
  print('Limpe todos os datasets por favor antes de continuar')
else:
  print("Não existe dataset cadastrado! ")

Limpe todos os datasets por favor antes de continuar


In [7]:
# Create instance of s3
s3 = boto3.Session().resource('s3')
bucket = s3.Bucket(bucket_name)

In [8]:
object = s3.Object(bucket_name, 'your-file-in-s3')

path_to_download = "./your-path-to-download-the-file/name-of-file"

# Valida path to dataset
if (not os.path.isdir('./your-path')):
  os.mkdir('./your-path')
object.download_file(path_to_download)

data_c1 = pd.read_csv(path_to_download, names=['timestamp', 'item_id', 'demand'])

In [9]:
# Showing data

data_c1.rename({'demand': 'target_value'}, axis=1, inplace=True) ## Convert data to necessary!!!
data_c1.head(10)

,timestamp,item_id,target_value
0,timestamp,item_id,target_value
1,2003-10-13,78,820060.59
2,2000-07-13,65,302238.13
3,2000-09-28,95,127893.7
4,2001-07-15,30,75608.62
5,2002-09-13,81,873172.35
6,2003-01-02,83,367691.6
7,2000-04-13,86,450659.45
8,2001-01-06,47,606058.7
9,2002-09-14,14,1108263.02


# Create dataset

In [10]:
# Create role in aws iam

role_name = 'ForecastNotBook-basic'
iam = boto3.client('iam')
role_arn = iam.get_policy(PolicyArn='your-url-to-policy')

print(f'Policy ID {role_arn}')

Policy ID {'Policy': {'PolicyName': 'forecast-data-test-policy', 'PolicyId': 'ANPAXR4XQLJKIZRALGARK', 'Arn': 'arn:aws:iam::519471913556:policy/forecast-data-test-policy', 'Path': '/', 'DefaultVersionId': 'v43', 'AttachmentCount': 1, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2023, 3, 29, 15, 48, 26, tzinfo=tzlocal()), 'UpdateDate': datetime.datetime(2023, 3, 30, 14, 52, 54, tzinfo=tzlocal()), 'Tags': []}, 'ResponseMetadata': {'RequestId': '1bf52225-b25c-4f7b-a987-13fb678e907c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1bf52225-b25c-4f7b-a987-13fb678e907c', 'content-type': 'text/xml', 'content-length': '788', 'date': 'Fri, 31 Mar 2023 14:08:11 GMT'}, 'RetryAttempts': 0}}


In [11]:

def create_response():
  return forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn)

def create_predictor():
  return forecast.describe_auto_predictor(PredictorArn=predictor_arn)

# Responsable to clear all datasets, groups and groupsImports
def getAllGroupsAndDatasets():
  datasets = forecast.list_datasets()
  groupsImport = forecast.list_dataset_import_jobs()
  groups = forecast.list_dataset_groups()


  for group in groupsImport['DatasetImportJobs']:
    try:
      forecast.delete_dataset_import_job(DatasetImportJobArn=group['DatasetImportJobArn'])
    except Exception as e:
      print(e)

  for group in groups['DatasetGroups']:
    try:
      forecast.delete_dataset_group(DatasetGroupArn=group['DatasetGroupArn'])
    except Exception as e:
      print(e)

  for dataset in datasets['Datasets']:
    try:
      forecast.delete_dataset(DatasetArn=dataset['DatasetArn'])
    except Exception as e:
      print(e)

In [12]:
getAllGroupsAndDatasets()

An error occurred (ResourceInUseException) when calling the DeleteDatasetImportJob operation: Operation not allowed when the resource arn:aws:forecast:us-west-2:519471913556:dataset-import-job/C1_DATASET_5/C1 is being referenced by these resources : [ arn:aws:forecast:us-west-2:519471913556:predictor/C1_PREDICTOR_01GWVFJ2S8MB5GVXVVKB9GQYR3 ]
An error occurred (ResourceInUseException) when calling the DeleteDatasetGroup operation: Operation not allowed when the resource arn:aws:forecast:us-west-2:519471913556:dataset-group/DatasetC111 is being referenced by these resources : [ arn:aws:forecast:us-west-2:519471913556:predictor/C1_PREDICTOR_01GWVFJ2S8MB5GVXVVKB9GQYR3 ]
An error occurred (ResourceInUseException) when calling the DeleteDataset operation: Operation not allowed when the resource arn:aws:forecast:us-west-2:519471913556:dataset/C1_DATASET_5 is being referenced by these resources : [ arn:aws:forecast:us-west-2:519471913556:dataset-import-job/C1_DATASET_5/C1 ]


# Clean datasets

In [95]:
DATASET_FREQUENCY = 'H'
TS_DATASET_NAME = 'C1_DATASET_5'
TS_SCHEMA = {
    "Attributes": [
        {
            "AttributeName":'timestamp',
            "AttributeType":'timestamp',
        },
        {
            "AttributeName":'item_id',
            "AttributeType":'string',
        },
        {
            "AttributeName":'target_value',
            "AttributeType":'float'
        }
    ]
}

create_data_response = forecast.create_dataset(
  Domain='CUSTOM',
  DatasetType='TARGET_TIME_SERIES',
  DatasetName=TS_DATASET_NAME,
  DataFrequency=DATASET_FREQUENCY,
  Schema=TS_SCHEMA
)

In [96]:
ts_dtaset_arn = create_data_response['DatasetArn']
describe_dataset_res = forecast.describe_dataset(DatasetArn=ts_dtaset_arn)

In [97]:
print(f"The Dataset with ARN {ts_dtaset_arn} is now {describe_dataset_res['Status']}.")
key = 'your-file'
ts_s3_path = f"s3://{bucket_name}/{key}"

The Dataset with ARN arn:aws:forecast:us-west-2:519471913556:dataset/C1_DATASET_5 is now ACTIVE.


# List Roles in politics

In [98]:
policy_name_arn = role_arn['Policy']['Arn']

In [99]:
role =  iam.get_role(RoleName='AmazonForecast-ExecutionRole-1680013362325')
role_name = role['Role']['Arn']
print(f'Role name is {role_name}')

Role name is arn:aws:iam::519471913556:role/service-role/AmazonForecast-ExecutionRole-1680013362325


In [100]:
# Importing the dataset
TIMESTAMP_FORMAT = 'yyyy-MM-dd' # Type of timestamp what the use in dataset
TS_IMPORT_JOB_NAME = 'C1' ## This name can it change
TIMEZONE = 'EST' # Region of data use

ts_dataset_import_job_response = \
  forecast.create_dataset_import_job(
      DatasetImportJobName=TS_IMPORT_JOB_NAME,
      DatasetArn=ts_dtaset_arn,
      DataSource={
          "S3Config": {
              "Path": ts_s3_path,
              "RoleArn": role_name
              
          },
          
      },
      TimestampFormat=TIMESTAMP_FORMAT,
      TimeZone = TIMEZONE
  )


In [101]:
ts_dataset_import_job_arn = ts_dataset_import_job_response['DatasetImportJobArn']
describe_dataset_import_job_response = forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn)

print(f"Waiting for Dataset Import Job with ARN {ts_dataset_import_job_arn} to become ACTIVE. This process could take 5-10 minutes.\n\nCurrent Status:")
describe_dataset_import_job_response = forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn)
print(f"\n\nThe Dataset Import Job with ARN {ts_dataset_import_job_arn} is now {describe_dataset_import_job_response['Status']}.")

Waiting for Dataset Import Job with ARN arn:aws:forecast:us-west-2:519471913556:dataset-import-job/C1_DATASET_5/C1 to become ACTIVE. This process could take 5-10 minutes.

Current Status:


The Dataset Import Job with ARN arn:aws:forecast:us-west-2:519471913556:dataset-import-job/C1_DATASET_5/C1 is now CREATE_PENDING.


In [104]:
while create_response()['Status'] == 'CREATE_IN_PROGRESS':
  print('Criando dataset')
  sleep(1)

In [105]:
# Create a datagroup

DATASET_GROUP_NAME = "name-of-to-database"
DATASET_ARNS = [ts_dtaset_arn]

create_dataset_group_response = \
    forecast.create_dataset_group(Domain="CUSTOM",
                                  DatasetGroupName=DATASET_GROUP_NAME,
                                  DatasetArns=DATASET_ARNS)

In [106]:
dataset_group_arn = create_dataset_group_response['DatasetGroupArn']
describe_dataset_group_response = forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)

print(f"The DatasetGroup with ARN {dataset_group_arn} is now {describe_dataset_group_response['Status']}.")

The DatasetGroup with ARN arn:aws:forecast:us-west-2:519471913556:dataset-group/DatasetC111 is now ACTIVE.


# Train Predictor

In [109]:
PREDICTOR_NAME = "name-of-to-predictor"
FORECAST_HORIZON = 0 # points that use in predictor
FORECAST_FREQUENCY = "H"
HOLIDAY_DATASET = [{
        'Name': 'holiday',
        'Configuration': {
        'CountryCode': ['US']
    }
}]

create_auto_predictor_response = \
    forecast.create_auto_predictor(PredictorName = PREDICTOR_NAME,
                                   ForecastHorizon=FORECAST_HORIZON,
                                   ForecastFrequency = FORECAST_FREQUENCY,
                                   DataConfig = {
                                       'DatasetGroupArn': dataset_group_arn, 
                                       'AdditionalDatasets': HOLIDAY_DATASET
                                    },
                                   ExplainPredictor = False)

predictor_arn = create_auto_predictor_response['PredictorArn']
print(f"Waiting for Predictor with ARN {predictor_arn} to become ACTIVE. Depending on data size and predictor setting，it can take several hours to be ACTIVE.")

describe_auto_predictor_response = forecast.describe_auto_predictor(PredictorArn=predictor_arn)
print(f"\n\nThe Predictor with ARN {predictor_arn} is now {describe_auto_predictor_response['Status']}.")

Waiting for Predictor with ARN arn:aws:forecast:us-west-2:519471913556:predictor/C1_PREDICTOR_01GWTENBH8Y7J56S2RC5WEVRK6 to become ACTIVE. Depending on data size and predictor setting，it can take several hours to be ACTIVE.

Current Status:


The Predictor with ARN arn:aws:forecast:us-west-2:519471913556:predictor/C1_PREDICTOR_01GWTENBH8Y7J56S2RC5WEVRK6 is now CREATE_PENDING.


In [110]:
while create_predictor()['Status'] == 'CREATE_IN_PROGRESS':
  print('Criando dataset')
  sleep(1)